###IA376 - Introdução ao Aprendizado Profundo
###Universidade Estadual de Campinas

# Abstractive Summarization of Portuguese Texts by fine-tuning the Portuguese-Based T5 Model
###Caio Simão Villela<br>
RA 168342

For more information regarding the development of the project and results, please refer to the PDF file on the repository.

This project relied highly on open-source projects developed by the community, as well as other publications on the matter. Please refer to them on the pdf file as well.


## Imports and Installs

In [ ]:
!pip install git+https://github.com/huggingface/transformers
!pip install sentencepiece --quiet
!pip install datasets --quiet
!pip install pyarrow --quiet
!pip install pytorch_lightning==0.9.0 -q
!pip install nlp

import argparse
import glob
import os
import json
import time
import logging
import random
import re
from itertools import chain
from string import punctuation

import nltk
import numpy as np
nltk.download('punkt')

import gc
import torch
from transformers import T5Tokenizer, T5Model, T5ForConditionalGeneration
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer

from transformers import AdamW, get_linear_schedule_with_warmup

import pytorch_lightning as pl

from torch.utils.data import Dataset, DataLoader

from datasets import load_dataset, load_metric
# from nlp import load_metric

import pandas as pd

from pytorch_lightning.loggers import WandbLogger

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-s4q420xr
  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-s4q420xr
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done


[nltk_data] Downloading package punkt to /home/jupyter/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


### wandb logger

In [ ]:
# Install Weights and Biases
!pip install wandb --upgrade -q

# Import wandb
import wandb

# Login with your authentication key
wandb.login()

# setup wandb environment variables
%env WANDB_PROJECT="t5base-summarizer-overfit"
%env WAND_WATCH='all'

wandb_logger = WandbLogger(project='t5-xsum')

wandb: Currently logged in as: cvillela (use `wandb login --relogin` to force relogin)


env: WANDB_PROJECT="t5base-summarizer-overfit"
env: WAND_WATCH='all'


# Utils

In [ ]:
checkpoint = 'unicamp-dl/ptt5-base-portuguese-vocab'

tokenizer = T5Tokenizer.from_pretrained(checkpoint)

In [ ]:
raw_datasets = load_dataset("xsum")

Using custom data configuration default
Reusing dataset xsum (/home/jupyter/.cache/huggingface/datasets/xsum/default/1.2.0/4957825a982999fbf80bca0b342793b01b2611e021ef589fb7c6250b3577b499)


In [ ]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(42)

# DATASET

In [1]:
class XSumDataset(Dataset):
    def __init__(self, tokenizer, type_path, num_samples, max_input_length, max_target_length, print_text=False):
        
        self.type_path = type_path
        self.dataset =  raw_datasets[type_path]
        self.max_input_length = max_input_length
        self.tokenizer = tokenizer
        self.max_target_length = max_target_length
        self.print_text = print_text
        self.prefix = "summarize: "
        if num_samples:
            self.dataset = self.dataset.select(list(range(0, num_samples)))
        
    def __len__(self):
        return self.dataset.shape[0]
    
    def clean_text(self, text):
        text = text.replace('\n','')
        text = text.replace('``', '')
        text = text.replace('"', '')
        
        return text
    
    
    def convert_to_features(self, example_batch):
        # Tokenize contexts and questions (as pairs of inputs)
        
        if self.print_text:
            print("Input Text: ", self.clean_text(example_batch['document']))

        
        input_ = self.prefix + self.clean_text(example_batch['document'])
        target_ = self.clean_text(example_batch['summary'])
        
        source = self.tokenizer.batch_encode_plus([input_], max_length=self.max_input_length, 
                                                     padding='max_length', truncation=True, return_tensors="pt")
        
        targets = self.tokenizer.batch_encode_plus([target_], max_length=self.max_target_length, 
                                                     padding='max_length', truncation=True, return_tensors="pt")
       
        return source, targets
  
    def __getitem__(self, index):
        source, targets = self.convert_to_features(self.dataset[index])
        
        source_ids = source["input_ids"].squeeze()
        target_ids = targets["input_ids"].squeeze()

        src_mask    = source["attention_mask"].squeeze()
        target_mask = targets["attention_mask"].squeeze()

        return {"source_ids": source_ids, "source_mask": src_mask, "target_ids": target_ids, "target_mask": target_mask}
        
  


NameError: ignored

In [ ]:
def get_dataset(tokenizer, type_path, num_samples, args):
      return XSumDataset(tokenizer=tokenizer, type_path=type_path, num_samples=num_samples,  max_input_length=args.max_input_length, 
                        max_target_length=args.max_output_length)

### Dataset class sanitycheck

In [ ]:
dataset = XSumDataset(tokenizer, 'train', 32, 512, 100, True)
len(dataset)

32

In [ ]:
data = dataset[31]
print()
print("Shape of Tokenized Text: ", data['source_ids'].shape)
print()
print("Sanity check - Decode Text: ", tokenizer.decode(data['source_ids'], skip_special_tokens=True))
print("====================================")
print("Sanity check - Decode Summary: ", tokenizer.decode(data['target_ids'], skip_special_tokens=True))

Input Text:  The win keeps the Candystripes two points behind leaders Dundalk who won 2-0 away to Shamrock Rovers.Former Plymouth striker Patterson scored his sixth goal of the season in the 14th minute at the Brandywell.He shot into an empty net after the ball broke to him when keeper Dean Delany thwarted Barry McNamee.Kurtis Byrne should have netted a speedy equaliser but the son of former Celtic player Paul Byrne completely missed his kick in front of goal.That was the one big scare for Kenny Shiels' men on a night when both keepers had a quiet night.Derry City have won six and drawn two in the eight games they have played since losing to Finn Harps on the first day of the season.

Shape of Tokenized Text:  torch.Size([512])

Sanity check - Decode Text:  summarize: The win keeps the Candystripes two points behind leaders Dundalk who won 2-0 away to Shamrock Rovers.Former Plymouth striker Patterson scored his sixth goal of the season in the 14th minute at the Brandywell.He shot into 

# Lightning Module

In [ ]:
args_dict = dict(
    output_dir="./test-summarization/", # path to save the checkpoints
    model_name_or_path=checkpoint,
    tokenizer_name_or_path=checkpoint,
    max_input_length=512,
    max_output_length=100,
    freeze_encoder=False,
    freeze_embeds=False,
    learning_rate=3e-4,
    weight_decay=1e-5,
    adam_epsilon=1e-8,
    warmup_steps=0,
    train_batch_size=4,
    eval_batch_size=4,
    num_train_epochs=2,
    gradient_accumulation_steps=32,
    n_gpu=1,
    resume_from_checkpoint=None, 
    val_check_interval = 0.05, 
    n_val=1000,
    n_train=-1,
    n_test=-1,
    early_stop_callback=False,
    fp_16=False, 
    opt_level='O1',
    max_grad_norm=1.0, 
    seed=42,
)

args = argparse.Namespace(**args_dict)


args_dict_en = dict(
    output_dir="./test-summarization/", # path to save the checkpoints
    model_name_or_path="t5-base",
    tokenizer_name_or_path="t5-base",
    max_input_length=512,
    max_output_length=100,
    freeze_encoder=False,
    freeze_embeds=False,
    learning_rate=3e-4,
    weight_decay=1e-5,
    adam_epsilon=1e-8,
    warmup_steps=0,
    train_batch_size=4,
    eval_batch_size=4,
    num_train_epochs=3,
    gradient_accumulation_steps=64,
    n_gpu=1,
#     resume_from_checkpoint='./test-summarization/checkpointepoch=1.ckpt',
    resume_from_checkpoint=None, 
    val_check_interval = 0.05, 
    n_val=-1,
    n_train=-1,
    n_test=-1,

    early_stop_callback=False,
    fp_16=False,
    opt_level='O1',
    max_grad_norm=1.0,
    seed=42,
)

args = argparse.Namespace(**args_dict)
args_en = argparse.Namespace(**args_dict_en)

print(args_dict_en)

{'output_dir': './test-summarization/', 'model_name_or_path': 't5-base', 'tokenizer_name_or_path': 't5-base', 'max_input_length': 512, 'max_output_length': 100, 'freeze_encoder': False, 'freeze_embeds': False, 'learning_rate': 0.0003, 'weight_decay': 1e-05, 'adam_epsilon': 1e-08, 'warmup_steps': 0, 'train_batch_size': 4, 'eval_batch_size': 4, 'num_train_epochs': 3, 'gradient_accumulation_steps': 64, 'n_gpu': 1, 'resume_from_checkpoint': None, 'val_check_interval': 0.05, 'n_val': -1, 'n_train': -1, 'n_test': -1, 'early_stop_callback': False, 'fp_16': False, 'opt_level': 'O1', 'max_grad_norm': 1.0, 'seed': 42}


# Logger

In [ ]:
logger = logging.getLogger(__name__)

class LoggingCallback(pl.Callback):
    def on_validation_end(self, trainer, pl_module):
        logger.info("***** Validation results *****")
        if pl_module.is_logger():
            metrics = trainer.callback_metrics
            # Log results
            for key in sorted(metrics):
                if key not in ["log", "progress_bar"]:
                    logger.info("{} = {}\n".format(key, str(metrics[key])))

    def on_test_end(self, trainer, pl_module):
        logger.info("***** Test results *****")

        if pl_module.is_logger():
            metrics = trainer.callback_metrics

            # Log and save results to file
            output_test_results_file = os.path.join(pl_module.hparams.output_dir, "test_results.txt")
            with open(output_test_results_file, "w") as writer:
                for key in sorted(metrics):
                    if key not in ["log", "progress_bar"]:
                        logger.info("{} = {}\n".format(key, str(metrics[key])))
                        writer.write("{} = {}\n".format(key, str(metrics[key])))

In [ ]:
## Define Checkpoint function
checkpoint_callback = pl.callbacks.ModelCheckpoint(
    filepath=args.output_dir, prefix="checkpoint", monitor="val_loss", mode="min", save_top_k=3
)

## If resuming from checkpoint, add an arg resume_from_checkpoint
train_params = dict(
    accumulate_grad_batches=args.gradient_accumulation_steps,
    gpus=args.n_gpu,
    max_epochs=args.num_train_epochs,
    early_stop_callback=False,
    precision= 16 if args.fp_16 else 32,
    amp_level=args.opt_level,
    resume_from_checkpoint=args.resume_from_checkpoint,
    gradient_clip_val=args.max_grad_norm,
    checkpoint_callback=checkpoint_callback,
    val_check_interval=args.val_check_interval,
    logger=wandb_logger,
    callbacks=[LoggingCallback()],
)

/opt/conda/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:37: UserWarning: Checkpoint directory ./test-summarization/ exists and is not empty with save_top_k != 0.All files in this directory will be deleted when a checkpoint is saved!
  warnings.warn(*args, **kwargs)


# Pytorch Lightning Module

In [ ]:
class T5FineTuner(pl.LightningModule):
    def __init__(self, hparams):
        super(T5FineTuner, self).__init__()
        self.hparams=hparams    
        self.model = T5ForConditionalGeneration.from_pretrained(hparams.model_name_or_path)
        self.tokenizer = T5Tokenizer.from_pretrained(hparams.tokenizer_name_or_path)
        self.rouge_metric = load_metric('rouge')
        self.save_hyperparameters()
        
        if self.hparams.freeze_embeds:
            self.freeze_embeds()
        if self.hparams.freeze_encoder:
            self.freeze_params(self.model.get_encoder())
            assert_all_frozen(self.model.get_encoder())
            
            
        n_observations_per_split = {
            "train": self.hparams.n_train,
            "validation": self.hparams.n_val,
            "test": self.hparams.n_test,
        }
        self.n_obs = {k: v if v >= 0 else None for k, v in n_observations_per_split.items()}
        
    
    def freeze_params(self, model):
        for par in model.parameters():
            par.requires_grad = False
            
            
    def freeze_embeds(self):
        """Freeze token embeddings and positional embeddings for bart, just token embeddings for t5."""
        try:
            self.freeze_params(self.model.model.shared)
            for d in [self.model.model.encoder, self.model.model.decoder]:
                freeze_params(d.embed_positions)
                freeze_params(d.embed_tokens)
        except AttributeError:
            self.freeze_params(self.model.shared)
            for d in [self.model.encoder, self.model.decoder]:
                self.freeze_params(d.embed_tokens)
    
    def lmap(self, f, x):
        """list(map(f, x))"""
        return list(map(f, x))
    

    def is_logger(self):
        return self.trainer.proc_rank <= 0
    
    
    def parse_score(self, result):
        return {k: round(v.mid.fmeasure * 100, 4) for k, v in result.items()}
        
    def forward(
      self, input_ids, attention_mask=None, decoder_input_ids=None, decoder_attention_mask=None, lm_labels=None
  ):
        return self.model(
            input_ids,
            attention_mask=attention_mask,
            decoder_input_ids=decoder_input_ids,
            decoder_attention_mask=decoder_attention_mask,
            labels=lm_labels,
    )

    def _step(self, batch):
        lm_labels = batch["target_ids"]
        lm_labels[lm_labels[:, :] == self.tokenizer.pad_token_id] = -100

        outputs = self(
            input_ids=batch["source_ids"],
            attention_mask=batch["source_mask"],
            lm_labels=lm_labels,
            decoder_attention_mask=batch['target_mask']
        )

        loss = outputs[0]

        return loss
    
    
    def ids_to_clean_text(self, generated_ids):
        gen_text = self.tokenizer.batch_decode(
            generated_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True
        )
        return self.lmap(str.strip, gen_text)
    
    
    def _generative_step(self, batch) :
        
        t0 = time.time()
        
        generated_ids = self.model.generate(
            batch["source_ids"],
            attention_mask=batch["source_mask"],
            use_cache=True,
            decoder_attention_mask=batch['target_mask'],
            max_length=100, 
            num_beams=2,
            repetition_penalty=2.5, 
            length_penalty=1.0, 
            early_stopping=True
        )
        preds = self.ids_to_clean_text(generated_ids)
        target = self.ids_to_clean_text(batch["target_ids"])
            
        gen_time = (time.time() - t0) / batch["source_ids"].shape[0]  
    
        loss = self._step(batch)
        base_metrics = {'val_loss': loss}

        summ_len = np.mean(self.lmap(len, generated_ids))
        base_metrics.update(gen_time=gen_time, gen_len=summ_len, preds=preds, target=target)
        self.rouge_metric.add_batch(predictions=preds, references=target)
        
        return base_metrics
    

    def training_step(self, batch, batch_idx):
        loss = self._step(batch)

        tensorboard_logs = {"train_loss": loss}
        return {"loss": loss, "log": tensorboard_logs}
  
    def training_epoch_end(self, outputs):
        avg_train_loss = torch.stack([x["loss"] for x in outputs]).mean()
        tensorboard_logs = {"avg_train_loss": avg_train_loss}
        return {"avg_train_loss": avg_train_loss, "log": tensorboard_logs, 'progress_bar': tensorboard_logs}

    def validation_step(self, batch, batch_idx):
        return self._generative_step(batch)
  
    def validation_epoch_end(self, outputs):
        
        avg_loss = torch.stack([x["val_loss"] for x in outputs]).mean()
        tensorboard_logs = {"val_loss": avg_loss}
        
        rouge_results = self.rouge_metric.compute() 
        rouge_dict = self.parse_score(rouge_results)
    
        tensorboard_logs.update(rouge1=rouge_dict['rouge1'], rougeL=rouge_dict['rougeL'])
        
        ## Clear out the lists for next epoch
        self.target_gen= []
        self.prediction_gen=[]
        return {"avg_val_loss": avg_loss, 
                "rouge1" : rouge_results['rouge1'],
                "rougeL" : rouge_results['rougeL'],
                "log": tensorboard_logs, 'progress_bar': tensorboard_logs}

    def configure_optimizers(self):
        "Prepare optimizer and schedule (linear warmup and decay)"

        model = self.model
        no_decay = ["bias", "LayerNorm.weight"]
        optimizer_grouped_parameters = [
            {
                "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
                "weight_decay": self.hparams.weight_decay,
            },
            {
                "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
                "weight_decay": 0.0,
            },
        ]
        optimizer = AdamW(optimizer_grouped_parameters, lr=self.hparams.learning_rate, eps=self.hparams.adam_epsilon)
        self.opt = optimizer
        return [optimizer]
  
    def optimizer_step(self, epoch, batch_idx, optimizer, optimizer_idx, second_order_closure=None, using_native_amp=False):
        if self.trainer.use_tpu:
            xm.optimizer_step(optimizer)
        else:
            optimizer.step()
        optimizer.zero_grad()
        self.lr_scheduler.step()
  
    def get_tqdm_dict(self):
        tqdm_dict = {"loss": "{:.3f}".format(self.trainer.avg_loss), "lr": self.lr_scheduler.get_last_lr()[-1]}

        return tqdm_dict
    

    def train_dataloader(self):   
        n_samples = self.n_obs['train']
        train_dataset = get_dataset(tokenizer=self.tokenizer, type_path="train", num_samples=n_samples, args=self.hparams)
        dataloader = DataLoader(train_dataset, batch_size=self.hparams.train_batch_size, drop_last=True, shuffle=True, num_workers=4)
        t_total = (
            (len(dataloader.dataset) // (self.hparams.train_batch_size * max(1, self.hparams.n_gpu)))
            // self.hparams.gradient_accumulation_steps
            * float(self.hparams.num_train_epochs)
        )
        scheduler = get_linear_schedule_with_warmup(
            self.opt, num_warmup_steps=self.hparams.warmup_steps, num_training_steps=t_total
        )
        self.lr_scheduler = scheduler
        return dataloader

    def val_dataloader(self):
        n_samples = self.n_obs['validation']
        validation_dataset = get_dataset(tokenizer=self.tokenizer, type_path="validation", num_samples=n_samples, args=self.hparams)
        
        return DataLoader(validation_dataset, batch_size=self.hparams.eval_batch_size, num_workers=4)
    
    def test_step(self, batch, batch_idx):
        return self._generative_step(batch)
    
    def test_dataloader(self):
        n_samples = self.n_obs['test']
        test_dataset = get_dataset(tokenizer=self.tokenizer, type_path="test", num_samples=n_samples, args=self.hparams)
        
        return DataLoader(test_dataset, batch_size=self.hparams.eval_batch_size, num_workers=4)

# Training

In [ ]:
model = T5FineTuner(args)

In [ ]:
trainer = pl.Trainer(**train_params)

In [ ]:
trainer.fit(model)

In [ ]:
trainer.save_checkpoint("ptt5-base-finetune-2.ckpt")

# Testing and Validating

In [ ]:
import textwrap
from tqdm.auto import tqdm

### Instantiate model from checkpoint

In [ ]:
wandb_logger = WandbLogger(project='t5-xsum-TEST')

In [ ]:
model_pt = T5FineTuner(args) 
model_pt.load_state_dict(torch.load("ptt5-base-finetune.ckpt")['state_dict'])

<All keys matched successfully>

In [ ]:
model_en = T5FineTuner(args_en) 
model_en.load_state_dict(torch.load("t5-base-finetune.ckpt")['state_dict'])

<All keys matched successfully>

In [ ]:
checkpoint_pt = 'unicamp-dl/ptt5-base-portuguese-vocab'
checkpoint_en = 't5-base'

tokenizer_pt = T5Tokenizer.from_pretrained(checkpoint_pt)
tokenizer_en = T5Tokenizer.from_pretrained(checkpoint_en)

### Summary Generation

In [ ]:
def predict_from_text(text, model, tokenizer, max_input_length=512, max_summary_length=100, min_summary_length=30, length_penalty = 15, n_gram = 2, n_beams = 2, n_sequence = 1 ):
    
    model.to('cpu')

    tokens_input = tokenizer.encode("summarize: " + text,
                                  return_tensors='pt',
                                  max_length=max_input_length,
                                    padding='max_length',
                                  truncation=True)

    summary_ids = model.model.generate(tokens_input,
                                early_stopping= True,
                                length_penalty = 1,
                                repetition_penalty = 2.5,
                                max_length = 150,
                                min_length = min_summary_length,
                                no_repeat_ngram_size = n_gram,
                                num_beams = 2,
                                num_return_sequences = n_sequence
                                )
    
    print(140*'-')
    print('input length: ', len(tokens_input[0]))
    
    for summa in summary_ids:
        print('predicted summary length: ', len(summa))
        print("PREDICTED SUMMARY: ")
        summary = tokenizer.decode(summa, skip_special_tokens=True)
        print(summary)

In [ ]:
def predict_from_dataset(model, tokenizer):
    dataset = XSumDataset(tokenizer, 'test', 32, 512, 100, True)
    len(dataset)

    loader = DataLoader(dataset, batch_size=32, shuffle=True)
    it = iter(loader)
    batch = next(it)

    model.to('cuda')
    outs = model.model.generate(
                batch["source_ids"].cuda(),
                attention_mask=batch["source_mask"].cuda(),
                use_cache=True,
                decoder_attention_mask=batch['target_mask'].cuda(),
                max_length=150, 
                num_beams=2,
                repetition_penalty=2.5, 
                length_penalty=1.0, 
                early_stopping=True
            )

    dec = [tokenizer.decode(ids, skip_special_tokens=True) for ids in outs]
    texts = [tokenizer.decode(ids, skip_special_tokens=True) for ids in batch['source_ids']]
    targets = [tokenizer.decode(ids, skip_special_tokens=True) for ids in batch['target_ids']]
    
    for i in range(5):
        lines = textwrap.wrap("XSum Article:\n%s\n" % texts[i], width=100)
        print("\n".join(lines))
        print("\nActual Summary: %s" % targets[i])
        print("\nPredicted Summary: %s" % dec[i])
        print("=====================================================================\n")

In [ ]:
def predict_from_dataset_pt_en(model_pt, tokenizer_pt, model_en, tokenizer_en):
    dataset_pt = XSumDataset(tokenizer_pt, 'test', 32, 512, 100, True)
    loader_pt = DataLoader(dataset_pt, batch_size=32, shuffle=False)
    it_pt = iter(loader_pt)
    batch_pt = next(it_pt)

    model_pt.to('cuda')
    outs_pt = model_pt.model.generate(
                batch_pt["source_ids"].cuda(),
                attention_mask=batch_pt["source_mask"].cuda(),
                use_cache=True,
                decoder_attention_mask=batch_pt['target_mask'].cuda(),
                max_length=150, 
                num_beams=2,
                repetition_penalty=2.5, 
                length_penalty=1.0, 
                early_stopping=True
            )

    dec_pt = [tokenizer_pt.decode(ids, skip_special_tokens=True) for ids in outs_pt]

    
    dataset_en = XSumDataset(tokenizer_en, 'test', 32, 512, 100, True)
    loader_en = DataLoader(dataset_en, batch_size=32, shuffle=False)
    it_en = iter(loader_en)
    batch_en = next(it_en)
    
    model_en.to('cuda')
    outs = model_en.model.generate(
                batch_en["source_ids"].cuda(),
                attention_mask=batch_en["source_mask"].cuda(),
                use_cache=True,
                decoder_attention_mask=batch_en['target_mask'].cuda(),
                max_length=150, 
                num_beams=2,
                repetition_penalty=2.5, 
                length_penalty=1.0, 
                early_stopping=True
            )

    dec_en = [tokenizer_en.decode(ids, skip_special_tokens=True) for ids in outs]
    
    texts = [tokenizer_en.decode(ids, skip_special_tokens=True) for ids in batch_en['source_ids']]
    targets = [tokenizer_en.decode(ids, skip_special_tokens=True) for ids in batch_en['target_ids']]
    
    for i in range(20):
        lines = textwrap.wrap("XSum Article:\n%s\n" % texts[i], width=100)
        print("\n".join(lines))
        print("\nActual Summary: %s" % targets[i])
        print("\nPredicted Summary [T5-base]: %s" % dec_en[i])
        print("\nPredicted Summary [PT-T5]: %s" % dec_pt[i])
        print("=====================================================================\n")

### Test Dataset

In [ ]:
transito = """ 

Aumentou a média de lentidão no trânsito na cidade de São Paulo nesta terça-feira (6), segundo dados da CET (Companhia de Engenharia de Tráfego)

De 71 km de congestionamento na terça passada (29), a capital registrou 81 km nesta terça. O valor também é superior ao registrado duas semanas atrás (22), com 65 km.

Também aumentou o número de veículos circulando nas ruas da cidade, com cerca de 6,5 milhões nesta semana, 6,35 milhões na semana passada e 6,3 milhões na anterior.

Também houve um leve aumento no número de pessoas nos ônibus, de acordo com a SPTrans. Cerca de 1,93 milhão de passageiros foram transportados ontem, contra 1,87 milhão uma semana atrás e 1,88 milhão há duas semanas.
"""

butanvac = """A Anvisa (Agência Nacional de Vigilância Sanitária) autorizou nesta quarta-feira (7) o início da vacinação de voluntários que farão parte dos testes clínicos da vacina Butanvac, em desenvolvimento pelo Instituto Butantan contra a Covid-19.

A decisão foi tomada após reunião entre as equipes técnicas da agência e do laboratório.

A autorização para os testes já havia sido concedida em 9 de junho. Segundo a Anvisa, porém, ainda havia informações pendentes, as quais foram apresentadas nesta quarta.

Com isso, as doses já poderão ser aplicadas para os testes.

O estudo será realizado no Hospital das Clínicas da Faculdade de Medicina da Universidade de São Paulo (HCFMUSP) e no Hospital das Clínicas da Faculdade de Medicina de Ribeirão Preto (HCFMRP)."""

adianta = """

O governador João Doria (PSDB) disse nesta quarta (7) que o Instituto Butantan irá antecipar em um mês a entrega das 100 milhões de doses de Coronavac ao Ministério da Saúde.

O contrato previa a entrega até 30 de setembro, mas Doria afirmou que as doses estarão disponíveis até 31 de agosto.

O Butantan já entregou 53 milhões de doses e enviará mais 10 milhões na próxima semana. Ainda segundo Doria, no dia 14 de julho está prevista a chegada de mais 12 mil litros de insumo para a produção de 20 milhões de doses da Coronavac.

"Até 31 de agosto vamos cumprir integralmente o compromisso previsto inicialmente para o fim de setembro. Há senso de urgência para salvar vidas", disse o governador.

Doria também anunciou ter comprado 4 milhões de doses adicionais da Coronavac, diretamente com o laboratório chinês Sinovac. Destas, 2,7 milhões chegam nesta quarta-feira a São Paulo e até o dia 26 de julho o estado deverá receber o restante.

Segundo o governador, a compra adicional vai permitir a antecipação da vacinação em São Paulo. Não foi informado como será a antecipação do calendário de imunização.

"Com isso, vamos antecipar o calendário de vacinação em São Paulo, sem interferir no contrato do Butantan com o Ministério da Saúde", explicou.
"""

eua = """
O governo americano deve anunciar a doação de mais uma leva de vacinas contra Covid-19 ao Brasil nos próximos 15 dias. Pessoas que participam das negociações dizem que os imunizantes serão da Janssen, de aplicação única, ou da Pfizer, que demanda duas doses —ambos foram aprovados para uso nos dois países.

Negociadores afirmam que o envio deve ser feito via doação direta, mas ainda não há confirmação sobre a data e quantidade específica de doses que chegarão ao Brasil. Se for confirmada nos termos das atuais tratativas, essa será a segunda doação direta de vacinas feita pelo governo americano aos brasileiros.

Em 23 de junho, a Casa Branca anunciou o envio de 3 milhões de doses da Janssen, que já chegaram ao Brasil e começaram a ser aplicadas.

No dia 4 de julho, o presidente Jair Bolsonaro enviou uma carta a Joe Biden para agradecer pelo envio das vacinas e parabenizar os americanos pelo Dia da Independência do país, comemorado naquela data.

As novas vacinas devem ser despachadas saindo da Flórida, com chegada ao aeroporto de Viracopos, em Campinas (SP), com transporte feito pela companhia aérea Azul, que ofereceu o serviço ao governo brasileiro da primeira vez e poderia repetir o traslado. Caso a empresa não esteja disponível, o Brasil deve utilizar aeronaves da FAB (Força Aérea Brasileira).

As primeiras 3 milhões de doses foram o maior número de vacinas mandados pelos EUA para qualquer país até agora de forma direta —ou seja, fora do escopo do Covax, iniciativa vinculada à Organização Mundial da Saúde (OMS) para a distribuição de imunizantes a nações em desenvolvimento.

A negociação para a nova doação está sendo feita diretamente entre a Casa Branca e o governo brasileiro, que fez seu primeiro contato em busca de vacinas em março, depois do pedido de outros líderes da região, como o presidente mexicano.

O Brasil também tem se apresentado como possível receptor de cerca de 20 milhões de doses que estão nos estoques americanos e devem perder a validade em setembro. Esses imunizantes seriam distribuídos via Covax, mas o consórcio avisou que não deve ter a logística necessária para a distribuição dentro deste prazo. O governo brasileiro, então, comunicou aos EUA que pode receber ao menos parte dessas 20 milhões de doses.

Os americanos já anunciaram que cerca de 20 milhões de doses serão compartilhadas com países da América Latina e do Caribe nas próximas semanas, via Covax, parte do total de 80 milhões de doses prometidas por Joe Biden para diversos países do mundo.

O número para a região que inclui o Brasil foi considerado baixo diante dos 438 milhões de habitantes que vivem nos países latino-americanos e caribenhos.

Via Covax, o Brasil terá que dividir as doses com Argentina, Colômbia, Peru, Equador, Paraguai, Bolívia, Uruguai, Guatemala, El Salvador, Honduras, Haiti, República Dominicana, Panamá, Costa Rica e outras nações do Caribe.
"""

vereadores = """
Tema que tem sido investigado pela CPI no Senado, o incentivo ao uso de medicamentos sem comprovação científica para tratar da Covid-19 também entrou em cena em Câmaras de Vereadores da Grande São Paulo.

Baseados em notícias falsas sobre outros municípios, parlamentares de São Bernardo do Campo, Mogi das Cruzes e Osasco fizeram propostas para cobrar das respectivas prefeituras o que chamam de “tratamento precoce” contra a doença.

Medicamentos como ivermectina estão entre as solicitações dos políticos levadas às prefeituras. No entanto, não há evidência de eficácia desse tipo de remédio contra o vírus.
"""

sp38 = """
O prefeito de São Paulo, Ricardo Nunes (MDB), tratará em Brasília nesta terça-feira (6) da liberação da vacinação da população entre 12 e 17 anos, com foco no público com comorbidades.

O assunto será um entre vários temas tratados com o governo Jair Bolsonaro (sem partido).

Às 13h, Nunes e o secretário Edson Aparecido tratarão do tema com o ministro da Saúde, Marcelo Queiroga.

Os adolescentes ainda não fazem parte do plano nacional de imunização. No entanto, a gestão paulistana se apoia na autorização, pela Anvisa (Agência Nacional de Vigilância Sanitária), da vacinação deste público com a Pfizer no país.

Além disso, o Instituto Butantan informou que pediria autorização para aplicação da Coronavac em crianças, uma vez que o imunizante foi testado na China e se mostrou seguro.

Na área da saúde, a gestão municipal também deverá tratar da habilitação de leitos para a cidade.

Em uma série de agendas com ministros e autoridades, Nunes também tratará de liberação de verba para a construção do BRT (espécie de corredor de ônibus mais moderno) da avenida Aricanduva, na zona leste da capital, e também para obras de retrofit no centro da capital.

Às 15h30, Nunes se encontrará com o presidente Jair Bolsonaro. O presidente da Câmara Municipal de São Paulo, Milton Leite (DEM), também participará do encontro.
"""

In [ ]:
# Dataset selection
dataset_en = XSumDataset(tokenizer_en, 'test', 50, 512, 100, True)

In [ ]:
predict_from_text(sp38, model_pt, tokenizer_pt)

--------------------------------------------------------------------------------------------------------------------------------------------
input length:  512
predicted summary length:  31
PREDICTED SUMMARY: 
Police are comeback for the first time da gestão municipal na área da vacinação do público entre 12 e 17 anos.


In [ ]:
predict_from_dataset(model_en, tokenizer_en)

Input Text:  Sevilla, winners of the tournament in 2014 and 2015, led early on through Vitolo's close-range strike.Marlos equalised with a composed finish from 18 yards before Taras Stepanenko headed the Ukrainian side in front before half-time.But Kevin Gameiro's late penalty earned the Spaniards a draw.The French attacker showed no signs of nerves, converting confidently from the spot after Facundo Ferreyra fouled Vitolo just inside the area.The result leaves Sevilla on course for a third consecutive Europa League success, with the two away goals making them firm favourites to advance to the final in Basel.No team has won three consecutive European titles since Bayern Munich achieved the feat 40 years ago, and if Unai Emery's side progress from next Thursday's second leg in Spain, only Liverpool or Villarreal stand between them and a piece of history. The first leg of that tie ended 1-0 to Villarreal.Sevilla have not won away from home all season in La Liga and have managed just one 

/opt/conda/lib/python3.7/site-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448265233/work/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)


XSum Article: summarize: Sevilla, winners of the tournament in 2014 and 2015, led early on through
Vitolo's close-range strike.Marlos equalised with a composed finish from 18 yards before Taras
Stepanenko headed the Ukrainian side in front before half-time.But Kevin Gameiro's late penalty
earned the Spaniards a draw.The French attacker showed no signs of nerves, converting confidently
from the spot after Facundo Ferreyra fouled Vitolo just inside the area.The result leaves Sevilla on
course for a third consecutive Europa League success, with the two away goals making them firm
favourites to advance to the final in Basel.No team has won three consecutive European titles since
Bayern Munich achieved the feat 40 years ago, and if Unai Emery's side progress from next Thursday's
second leg in Spain, only Liverpool or Villarreal stand between them and a piece of history. The
first leg of that tie ended 1-0 to Villarreal.Sevilla have not won away from home all season in La
Liga and have manag

In [ ]:
predict_from_dataset(model_pt, tokenizer_pt)

Input Text:  The proposals mean charges would be placed on 625 spaces in the area and restrictions placed on a further 741.Reading Borough Council has spent four years designing the scheme to relieve parking pressures at the hospital and to make parking easier for residents.The petition, set up by hospital staff, describes the scheme as problematic.The online petition opposes the introduction of pay and display meters for Addington Road, Erleigh Road and other roads around the hospital in Reading.Clare Goulbourn-Lay, a Royal Berkshire Hospital midwife, set up the petition to express her concerns about the potential consequences of the scheme.By making changes to this road, you're not just making it difficult for the staff, you're making it difficult for the patients and pushing the problem further out, she said.With the university so close as well, there simply isn't enough parking space as it is. To add another restriction is just silly.She has called for the scheme to be cancelled or

In [ ]:
predict_from_dataset_pt_en(model_pt, tokenizer_pt, model_en, tokenizer_en)

Input Text:  Fast forward about 20 years, and it's fair to say he has done just that.The business he runs, Frasers Hospitality, is one of the world's biggest providers of high-end serviced apartments. Its 148 properties span about 80 capital cities, as well as financial hubs across Europe, Asia, the Middle East and Africa.But it almost didn't get off the ground.When Mr Choe was appointed to launch and lead the company, Asia was booming; the tiger economies of Hong Kong, South Korea, Taiwan and Singapore were expanding rapidly.But as Frasers prepared to open its first two properties in Singapore, the Asian financial crisis hit.It was 1997. Currencies went into freefall. Suddenly, people were losing their jobs and stopped travelling.Mr Choe recalls asking staff if they really wanted to continue working with the firm, because when the properties opened they might not get paid.It was really that serious, he says. I remember tearing up because they said 'let's open it, let's open it whether